# AB Testing

A Electronic House é um comercio online ( e-commerce ) de produtos de informática
para casas e escritórios. Os clientes podem comprar mouses, monitores, teclados,
computadores, laptops, cabos HDMI, fones de ouvido, cameras webcam, entre
outros, através de um site online e recebem os produtos no conforto de suas casas.

O time de UX designers vem trabalhando em uma nova página de vendas, com o
objetivo de aumentar a taxa de conversão de um produto da loja, um teclado bluetooth. O product manager (gerente de produto) disse que a taxa de conversão
da página atual é de 13% em média, no último ano.

O objetivo do product manager é aumentar a taxa de conversão em 2%, ou seja, a
nova página de vendas, desenvolvida pelo time de UX, seria um sucesso se a sua
taxa de conversão fosse de 15%.

O teclado bluetooth possui um preço de venda de R$ 4.500,00 à vista ou parcelado
em 12% sem juros no cartão de crédito.

Antes de trocar a página de vendas antiga pela nova, o product manager gostaria
de testar a efetividade da nova página em um grupo menor de clientes, a fim de
correr menos riscos de queda da conversão, caso a página nova mostre uma
conversão pior do que a página atual.

## 0.1. Imports

In [32]:
import math
import pandas as pd
import numpy as np

from statsmodels.stats import api as sm
from scipy.stats import chi2_contingency

# 1.0 Load data

In [2]:
data_raw = pd.read_csv('/datasets/ab_data.csv')

In [3]:
data_raw

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


# 2.0 Design de Experimentos

## 2.1 Formulação das Hipóteses

H0: A conversão da nova página é de 13%

H1: A conversão da nova página é diferente de 13%

## 2.2 Parâmetros do Experimento

In [4]:
# nível de confiança
confidence_level = 0.95

# nível de significância
significance_level = 0.05

# conversões da página nova e página atual
p1 = 0.15
p2 = 0.13

# tamanho do efeito
effect_size = sm.proportion_effectsize(p1, p2)

# poder estatístico
power = 0.80

In [5]:
# sample size
sample_n = math.ceil(sm.NormalIndPower().solve_power(
    effect_size,
    power=power,
    alpha=significance_level
))
sample_n

4720

In [6]:
print(f'O tamanho da amostra do a ser coletado de ambos os grupos é de :{sample_n}')
print(f'Tamanho total da amostra: {2*sample_n}')

O tamanho da amostra do a ser coletado de ambos os grupos é de :4720
Tamanho total da amostra: 9440


# 3.0 Análise descritiva dos dados

In [7]:
print(f'Number of rows: {data_raw.shape[0]}')      
print(f'Number of columns: {data_raw.shape[1]}')

Number of rows: 294478
Number of columns: 5


## Verificação dos dados faltantes

In [8]:
data_raw.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

## Conferir as "flags" do teste A

In [9]:
data_raw[['user_id', 'group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

,group,landing_page,user_id
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


## Removendo flags duplicadas

In [10]:
# grouping users flags and removing the ones with more than more
data_users_delete = data_raw[['user_id', 
                              'group']].groupby('user_id').count().reset_index().query('group > 1')['user_id']

# selecting only users with one flag
df1 = data_raw[~data_raw['user_id'].isin(data_users_delete)]

## Amostragem dos grupos de tratamento e controle

In [11]:
# Control group
df_control_sample = df1[df1['group'] == 'control'].sample(n=sample_n, random_state=7)
print(f'Size of Control Group :{df_control_sample.shape[0]}')

# Treatment group
df_treatment_sample = df1[df1['group'] == 'treatment'].sample(n=sample_n, random_state=7)
print(f'Size of Treatment Group: {df_treatment_sample.shape[0]}')

# Total sample
df_ab = pd.concat([df_control_sample, df_treatment_sample]).reset_index(drop=True)

Size of Control Group :4720
Size of Treatment Group: 4720


## Cálculo da métrica de interesse

In [13]:
# Control group convertion
conversion_rate_control = df_control_sample['converted'].mean()
print(f'Conversion Rate - Control Group : {conversion_rate_control}')

# Treatment group convertion
conversion_rate_treatment = df_treatment_sample['converted'].mean()
print(f'Conversion Rate - Treatment Group : {conversion_rate_treatment}')

Conversion Rate - Control Group : 0.12521186440677967
Conversion Rate - Treatment Group : 0.12033898305084746


# 4.0 Teste de Hipóteses

In [21]:
df_table = pd.crosstab(index=df_ab['group'], columns=df_ab['converted'])
df_table.columns = ['not_converted', 'converted']
df_table

,not_converted,converted
group,,
control,4129,591
treatment,4152,568


In [23]:
chi_val, pvalue, dof, expected = chi2_contingency(df_table)

print(f'p-value : {pvalue}')

if pvalue < significance_level:
    print('Rejeita a hipótese nula')
else:
    print('Falha em rejeitar a hipótese nula')

p-value : 0.49021768956704426
Falha em rejeitar a hipótese nula


Com os dados coletados não é possível afirmar que a diferença é significante.

As opções para seguir com o teste são:
* Coletar mais dados
* Melhorar o efeito aprimorando a página

# 5.0 Conversão da página em faturamento

Considerando o ticket de R$ 4.500,00, podemos verificar qual seria o faturamento caso o teste da página fosse bem sucedido.

In [24]:
df3 = df1.copy()

In [26]:
# converting timestamp to year-month-day format
df3['timestamp'] = pd.to_datetime(df3['timestamp']).apply(lambda x: x.strftime('%Y-%m-%d'))

df_date = df3[['user_id', 'timestamp']].groupby('timestamp').count().reset_index()

In [39]:
df_date['current_conversions'] = np.ceil(df_date['user_id']*0.13).astype(int)
df_date['current_GMV'] = df_date['current_conversions']*4500

df_date['new_conversions'] = np.ceil(df_date['user_id']*0.15).astype(int)
df_date['new_GMV'] = df_date['new_conversions']*4500

current_gmv = df_date['current_GMV'].sum()
new_gmv = df_date['new_GMV'].sum()

print('GMV on period: {}'.format(current_gmv))
print('New GMV on period: {}'.format(new_gmv))

lift_abs = new_gmv-current_gmv
lift = 100*(new_gmv-current_gmv)/current_gmv

print(f'Abs Lift: {lift_abs}')
print(f'Expected Lift: {lift:.2f}%')

GMV on period: 167760000
New GMV on period: 193563000
Abs Lift: 25803000
Expected Lift: 15.38%


Considerando o aumento da conversão para 15% poderíamos esperar um aumento de 15,38% no GMV.